# Finding the Ingredients of Other Worlds

##### By [Everett Schlawin](http://mips.as.arizona.edu/~schlawin/)

In this simulator you will be recovering spectra from exoplanet systems to understand what their atmospheres are made of.

# Table of Contents

* [How to Use This Notebook](#How-to-Use-This-Notebook)
* [1. Introduction - Planet Lightcurves](#1.-Introduction:-Planet-Lightcurves)
* [2. Planet Radius](#2.-Planet-Radius)
* [3. A Planet Spectrum](#3.-A-Planet-Spectrum)


# How to Use This Notebook

The webpage you are in is actually an app - much like the ones on your cellphone. This app consists of cells.

An *input* cell looks like a light grey box with an `In [ ]:` on its left. Input cells each contain code - instructions to make the computer do something.

To activate or select a cell, click anywhere inside of it.

<div class='alert alert-info'>
    <font size='3'><b>Select the cell below and read its contents.</b></font>
</div>

In [ ]:
# Text that follows a "#" is known as a comment.
# Comments do not affect your code in any way.
# You should always read the comments at the top of each cell you interact with.
# Comments will be used to describe what the cell's code is actually doing.

To execute or run a selected cell, hit `[Shift + Enter]` on your keyboard.

<div class='alert alert-info'>
    <font size='3'><b>Select the cell below and read its contents. Then, run the cell.</b></font>
</div>

In [ ]:
# Text that DOESN'T follow a "#" is considered code.
# Lines of code are instructions given to your computer.
# The line of code below is a "print" statement.
# A print statement literally prints out the text between its quotes.

print("Congrats! You have successfully run your first cell!")

Running a cell creates an *output* directly below it. An output can be some text, a graph, an interactive slider, or even nothing at all! For that last case, you know you have run a cell when the `In [ ]:` becomes `In [#]:`, where "#" is any number.

You can learn more about how Jupyter notebooks work at https://try.jupyter.org/
___

# 1. Introduction: Planet Lightcurves
Importing the necessary code


In [ ]:
import exo_engine

Start with an exoplanet transit lightcuve. This is a plot that shows how the brightness of a planet and star change with time as the planet goes in front of the star.

In [ ]:
exo_engine.plot_initial_lc()

Compare this to an animation from J Eastman (Penn State University) to see geometrically what is happening when a planet crosses in front of a star.

![SegmentLocal](HAT-P-3b-xpb0j3_eastman_psu.gif "segment")

# 2. Planet Radius
Experiment what happens at different radii and limb darkening. Does increasing the radius make the dip deeper or shallower? Below is an example with 2 different limb darkening parameters.

In [ ]:
rad = exo_engine.plot_interactive_rad()


# 3. A Planet Spectrum

And now look at the lightcurves for different colors

In [ ]:
from importlib import reload
reload(exo_engine)

exampleSpec = exo_engine.spectral_lc()
exampleSpec.plot_lc_multicolor()


Show the spectrum of the planet. What color will it's sillhoutte have?

In [ ]:
exampleSpec.spectrum_plot_i()



Visualize the surfaces of the planet at each color

In [ ]:
from importlib import reload ## This can be taken out in final version
reload(exo_engine) ## This can be taken out in final version
exampleSpec = exo_engine.spectral_lc()

exampleSpec.visualize_colors_i()



Now, what color do you think the sillhoutte will have?

## Now put in the star

In [ ]:
## Now put in a background star
fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect('equal')
ax.set_xlim(-1.3,1.3)
ax.set_ylim(-1.3,1.3)

circlePatch = plt.Circle((0., 0.), 10.,
                         edgecolor=one_color,facecolor='yellow')
ax.add_artist(circlePatch)
circlePatch = plt.Circle((0., 0.), np.min(radius_array) * 10.,
                         edgecolor='none',facecolor='black')
ax.add_artist(circlePatch)

sortInd = np.argsort(radius_array)
for ind in sortInd:
    thisRadius = radius_array[ind]
    for max_radius,one_color in zip(radius_array,colors_array):

        if thisRadius >= max_radius:
            circlePatch = plt.Circle((0., 0.), max_radius * 10.,linewidth=3,
                         edgecolor=one_color,facecolor='none',alpha=0.2)
            ax.add_artist(circlePatch)

ax.set_xlabel('X Size in Jupiters')
ax.set_ylabel('Y Size in Jupiters')


## An exoplanet Atmosphere with Molecules
Now interpret the lightcurves 

In [ ]:
dat = Table.read('opacity_breakdown_gto_f_hd189733b.fits')
for oneColumn in dat.colnames:
    if oneColumn != 'Wave':
        plt.semilogx(dat['Wave'],dat[oneColumn] * 100.,label=oneColumn)
plt.legend()
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Transit Depth (%)')

In [ ]:
dat.colnames

Look what happens when there are clouds: